# ENCODER DECODER MODEL FOR WORD LEVEL EMBEDDING

In [ ]:
## LOADING THE REQUIRED LIBRARIES
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import tensorflow as tf
from  tensorflow.keras.preprocessing.sequence import pad_sequences
from  sklearn.model_selection import train_test_split
from tqdm import tqdm

## Loading Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## LOADING THE PROCESSED DATASET  

df= pd.read_csv("/content/drive/MyDrive/ColabNotebooks/cs2/processed_data.csv")
df.columns = ["enc_input","dec_input","y"] 
df["dec_output"] = df.dec_input

## Adding start and end token

In [ ]:
## THE INPUTS TO THE DECODER REQUIRES SPECIAL TOKENS FOR THE START AND THE END SO WE ARE GOING TO USE 
## <start> AS BEGINING TOKEN
## <end>  AS END TOKEN

df["dec_input"]= "<start> " + df["dec_input"]
df["dec_output"] =  df["dec_output"] + " <end>" 

## Splitting And Sampling around 100k datapoints

---
##### THE TOTAL DATASET HAS 500K DATAPOINTS WHICH WILL TAKE MUCH HIGHER TRAINING TIME. THEREFORE I AM SAMPLING ONE-FIFTH OF THE TOTAL DATASET



In [ ]:
df_sampled = pd.concat((df[df.y==1].sample(frac= 0.2,random_state=1),df[df.y==2]))

In [ ]:
## ONCE THE DATA IS SAMPLED WE ARE SPLITTIND THE DATA IN TO TRAIN AND TEST

df_train ,df_val = train_test_split(df_sampled,test_size=0.2,random_state = 3, stratify = df_sampled.y )

In [ ]:
## IN THE COLUMN WHICH HAS DECODER INPUTS ADDING "<end>" TOKEN TO BE LEARNED BY THE TOKENIZER

df_train["dec_input"].iloc[0]  = df_train.iloc[0]["dec_input"] + " <end>"

In [ ]:
## HERE I AM SAMPLING 1000 POINTS FROM THE DATAFRAME AS TEST DATA WHICH ARE NOT PRESEENT IN THE TRAIN AND VALIDAION DATA
np.random.seed(5) 
df_test = df.loc[np.random.choice(np.array([x for x in df.index.values if x not in df_sampled.index.values]),1000,replace= False,)]

## Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
## TOKENIZER FOR ENCODER INPUT
tk_inp = Tokenizer()
tk_inp.fit_on_texts(df_train.enc_input.apply(str))

In [ ]:
# TOKENIZER FOR DECODER INPUT
tk_out = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n' )
tk_out.fit_on_texts(df_train.dec_input.apply(str))

## DATA PIPELINE

In [ ]:
## THIS CLASS CONVERTS TEXT DATA TO INTEGER SEQUENCES AND RETURNS THE PADDED SEQUENCES

class Dataset :
    def __init__(self, data , tk_inp ,tk_out, max_len):
        ## SETTING THE REQUIRED ATTRIBUTES
        self.encoder_inp = data["enc_input"].apply(str).values
        self.decoder_inp = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp
        self.tk_out = tk_out
        self.max_len = max_len
        
    def __getitem__(self,i):
        # INPUT SEQUENCES
        self.encoder_seq = self.tk_inp.texts_to_sequences([self.encoder_inp[i]])
        # DECODER INPUT SEQUENCES 
        self.decoder_inp_seq = self.tk_out.texts_to_sequences([self.decoder_inp[i]])
        # DECODER INPUT SEQUENCES
        self.decoder_out_seq = self.tk_out.texts_to_sequences([self.decoder_out[i]])
        
        # PADDING THE ENCODER INPUT SEQUENCES
        self.encoder_seq = pad_sequences(self.encoder_seq, padding="post",maxlen = self.max_len)
        # PADDING THE DECODER INPUT SEQUENCES
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, padding="post",maxlen = self.max_len)
        # PADDING DECODER OUTPUT SEQUENCES
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq ,padding="post", maxlen = self.max_len)

        ##  RETURNING THE ENCODER INPUT , DECODER INPUT , AND DECODER OUTPUT
        return self.encoder_seq ,  self.decoder_inp_seq,  self.decoder_out_seq
    
    def __len__(self):
        # RETURN THE LEN OF INPUT ENDODER
        return len(self.encoder_inp)

In [ ]:
## THIS CLASS CONVERTES THE DATASET INTO THE REQUIRED BATCH SIZE

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self,batch_size,dataset):
        # INTIALIZING THE REQUIRED VARIABLES 
        self.dataset = dataset
        self.batch_size = batch_size
        self.totl_points = self.dataset.encoder_inp.shape[0]
        
    def __getitem__(self,i):
        # STATING THE START AND STOP VATIABLE CONTAINGING INDEX VALUES FOR EACH BATCH
        start = i * self.batch_size
        stop = (i+1)*self.batch_size
        
        # PLACEHOLDERS FOR BATCHED DATA
        batch_enc =[]
        batch_dec_input = []
        batch_dec_out =[]

        for j in range(start,stop): 
            
            a,b,c = self.dataset[j] 
            batch_enc.append(a[0]) 
            batch_dec_input.append(b[0])
            batch_dec_out.append(c[0]) 
        
        # Conveting list to array   
        batch_enc = (np.array(batch_enc)) 
        batch_dec_input = np.array(batch_dec_input)
        batch_dec_out = np.array(batch_dec_out)
        
        ## RETURNING BATCHED DATA IN REQUIRED FORM
        return [batch_enc , batch_dec_input],batch_dec_out
    
    def __len__(self):
        # Returning the number of batches
        return int(self.totl_points/self.batch_size)

###### NOTE: WE ARE TAKING THE MAXIMUM LENGHT EQUAL TO 35 WHICH IS 99 PERCENTILE OF THE WORD LENGTH DISTRUBUTION

In [ ]:
# FORMING OBJECTS OF DATASET AND DATALOADER FOR TRAIN DATASET
train_dataset = Dataset(df_train,tk_inp,tk_out,35)
train_dataloader = Dataloader( batch_size = 512, dataset=train_dataset)

# FORMING OBJECTS OF DATASET AND DATALOADER FOR VALIDATION DATASET
val_dataset = Dataset(df_val , tk_inp,tk_out,35)
val_dataloader = Dataloader(batch_size=512 , dataset=val_dataset)

## Encoder Decoder Model

In [ ]:
## LOADING THE TENSORFLOW LIBRARIES

from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
## DEFINING THE ENCODER LAYER AS A FUNCTION

def encoder(input_shape,vocab, emb_output, lstm_units, enc_input):
    '''THIS FUNCTION TAKES IN THE SEQUENCES AND RETURNS THE ENCODER OUTPUT'''
    ## FIRST LAYER : EMBEDDING LAYER
    enc_emb = layers.Embedding(vocab, emb_output,mask_zero = True,input_length=input_shape)(enc_input)
    ## SECOND LAYER : LSTM LAYER
    enc_lstm , enc_state_h,enc_state_c = layers.LSTM(units= lstm_units,return_sequences=True,return_state=True)(enc_emb)
    ## RETURNING THE LSTM OUTPUTS AND STATES
    return enc_lstm , enc_state_h,enc_state_c


## DEFINING THE DECODER LAYER AS A FUNCTION 
def decoder(input_shape,vocab, emb_output, lstm_units,enc_states, dec_input):
  ## FIRST LAYER : EMBEDDING LAYER
  dec_emb = layers.Embedding(vocab, emb_output , mask_zero = True,input_length=input_shape)(dec_input)
  ## SECONG LAYER : LSTM LAYER
  dec_lstm, dec_state_h,dec_state_c = layers.LSTM(units=lstm_units,return_sequences=True,return_state=True)(dec_emb,initial_state= enc_states)
  ## RETURNING THE LSTM OUTPUTS AND STATES
  return dec_lstm, dec_state_h,dec_state_c

In [ ]:
## DEFINING THE MODEL ARCHITECTURE

# INPUT LAYER
enc_input = layers.Input(shape=(35))
# ENCODER DEFINED FORM FUNCTON ABOVE
enc_lstm , enc_state_h,enc_state_c = encoder(35,len(tk_inp.word_index)+1 , 300 ,256, enc_input )


# DECODER INPUT LAYER
dec_input = layers.Input(shape = (35))
# DECODER DEFINEA FROM ABOVE FUNCTION
dec_lstm , dec_state_h,dec_state_c = decoder(35,len(tk_out.word_index)+1 , 300 , 256 , [enc_state_h,enc_state_c],dec_input)
# DENCSE LAYER CONNECTOD TO DECODER OUTPUT
dense = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(dec_lstm)

# MODEL DEFINING
model  = Model(inputs=[enc_input,dec_input],outputs=dense)


In [ ]:
## DEFINING THE CALLBACKS
callback =[ tf.keras.callbacks.ModelCheckpoint( "/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_trainable_embedding/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001)
]

## STORING THE NUMBER OF STEPS IN ONE EPOCH FOR TRAIN AND VALIDATION DATASET
train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

# COMPILING THE MODEL
model.compile(optimizer="adam",loss='sparse_categorical_crossentropy')

In [ ]:
## FITTING THE MODEL
model.fit(train_dataloader,steps_per_epoch=train_steps,epochs=50,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback)

In [ ]:
# LOADING THE WEIGHTS FOR BEST MODEL
model.load_weights("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_trainable_embedding/besh.h5")

In [ ]:
## THIS FUNCTION IS USED IN THE INFERENCE TIME TO PREDICT THE RESULTS GIVEN THE INPUT TEXT

def predict(inp , model):
    ##  TAKES INPUT AS TEXT AND THE MODEL

    # CONVERT TEXT INPUT TO SEQUENCES 
    seq = tk_inp.texts_to_sequences([inp])
    # PADDING THE SEQUENCE
    seq = pad_sequences(seq,maxlen = 35,padding="post")
    ## INITIAL STATES FOR ENCODER
    state = [tf.zeros(shape=(1,256)),tf.zeros(shape= (1,256))]

    # SEQUENCE TO EMBEDDING
    enc_emb  = model.layers[2](seq)
    # PASSING EMBBEDDED SEQUENCES TO LSTM LAYER
    enc_output,state_h,state_c= model.layers[4](enc_emb,state)

    # PLACE HOLDER FOR PREDECTED WORDS
    pred = []
    # PLACE HOLDER FOR STATES 
    input_state = [state_h,state_c]
    # CURRENT VECTOR TO BE PASSED TO DECODER 
    current_vec = tf.ones((1,1))
    
    for i in range(35): # FOR i UP TO 35 (MAX LENGTH)
        ## CONVERT THE CURRENT VECTOR SEQUENCE WORD TO EMBEDDINGS
        dec_emb  = model.layers[3](current_vec)
        ## PASSING EMBEDDED VECTOR TO DECODER LSTM LAYER
        dec_output,dec_state_h,dec_state_c = model.layers[5](dec_emb , input_state)
        # PASSING DECODER OUTPUT TO DENSE LAYER
        dense = model.layers[6](dec_output)

        # SELECTING INDEX OF MAXIMUM DENSE OUTPUT AS CURRENT VECTOR
        current_vec = np.argmax(dense ,axis = -1)
        # UPDATING THE INPUT STATES
        input_state = [dec_state_h,dec_state_c]

        # APPENDING THE ACTUAL TEXT TO "pred" VARIABLE
        pred.append(tk_out.index_word[current_vec[0][0]])
        ## IF THE CURRENT VECTOR IS "<end>" BREAK THE LOOP
        if tk_out.index_word[current_vec[0][0]]=="<end>":
            break
    ## RETURN THE JOINED STRING IN LIST "pred"
    return " ".join(pred)

## PREDICTIONS ON TEST DATASET

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[19],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[50],model))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[50])

## Inference Time

In [ ]:
%%time
predict(df_test.enc_input.values[50],model)

## BELU SCORE 

In [ ]:
import nltk.translate.bleu_score as bleu

In [ ]:
# VALIDATION BELU SCORE
BLEU_val_emb = []
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model).split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_val_emb.append(b)
    except:
        continue

In [ ]:
print("Triain BELU Score = ",np.mean(BLEU_val_emb))

##  Model2 - Pre Trained Word Vector

In [ ]:
!wget --header="Host: doc-14-58-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://drive.google.com/" --header="Cookie: AUTH_oli2bj78otd6bj690sskhutcvt4jcljv_nonce=a17t06p88k2ng" --header="Connection: keep-alive" "https://doc-14-58-docs.googleusercontent.com/docs/securesc/l8ia6k4p0fn1kebbsov8ekehei507757/onk0h25n3rm35gh67s449ql1utt8rkp0/1628269950000/06848720943842814915/12599792132870778014/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&authuser=0&nonce=a17t06p88k2ng&user=12599792132870778014&hash=uvrme0lsn4ltpk6ivnbe741c9984e4ih" -c -O 'GoogleNews-vectors-negative300.bin.gz'

In [ ]:
## LOADING THE WORD2VEC MODEL 

from gensim import models
w = models.KeyedVectors.load_word2vec_format(r"GoogleNews-vectors-negative300.bin.gz", binary=True)

In [ ]:
## FORMING EMBEDDING MATRIX FOR INPUT SENTENCE
embedding_matrix_input = np.zeros((len(tk_inp.word_index)+1,300))

for word,index in tqdm(tk_inp.word_index.items()) :
    try:
        vec = w[word]
        embedding_matrix_input[index] = vec
    except:
      
        continue

In [ ]:
# FORMING EMBEDING MATRIX FOR OUTPUT SENTENCE
embedding_matrix_output = np.zeros((len(tk_out.word_index)+1,300))

for word,index in tqdm(tk_out.word_index.items()) :
    try:
        vec = w[word]
        embedding_matrix_output[index] = vec
    except:
   
        continue

## MODEL ARCHITECTURE

In [ ]:
def encoder2(input_shape,vocab, emb_output, lstm_units, enc_input,embedding_matrix):
  
  enc_emb = layers.Embedding(vocab, emb_output,weights= [embedding_matrix] ,mask_zero = True,input_length=input_shape,trainable=False)(enc_input)
  enc_lstm , enc_state_h,enc_state_c = layers.LSTM(units= lstm_units,return_sequences=True,return_state=True)(enc_emb)
  return enc_lstm , enc_state_h,enc_state_c

def decoder2(input_shape,vocab, emb_output, lstm_units,enc_states, dec_input,embedding_matrix):
  
  dec_emb = layers.Embedding(vocab, emb_output ,weights= [embedding_matrix], mask_zero = True,input_length=input_shape,trainable=False)(dec_input)
  dec_lstm, dec_state_h,dec_state_c = layers.LSTM(units=lstm_units,return_sequences=True,return_state=True)(dec_emb,initial_state= enc_states)
  return dec_lstm, dec_state_h,dec_state_c

In [ ]:
enc_input2 = layers.Input(shape=(35))
enc_lstm , enc_state_h,enc_state_c = encoder2(35,len(tk_inp.word_index)+1 , 300 ,256, enc_input2 ,embedding_matrix_input)

dec_input2 = layers.Input(shape = (35))
dec_lstm , dec_state_h,dec_state_c = decoder2(35,len(tk_out.word_index)+1 , 300 , 256 , [enc_state_h,enc_state_c],dec_input2,embedding_matrix_output)

dense2 = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(dec_lstm)

model2  = Model(inputs=[enc_input2,dec_input2],outputs=dense2)

In [ ]:
callback2 =[ tf.keras.callbacks.ModelCheckpoint( "/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_w2v/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001)
]

train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

model2.compile(optimizer="adam",loss='sparse_categorical_crossentropy')

In [ ]:
history = model2.fit(train_dataloader,steps_per_epoch=train_steps,epochs=50,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback2)

In [ ]:
model2.load_weights("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_w2v/besh.h5")

## PREDICTIONS FOR TEST DATASET

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[19],model2))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[50],model2))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[50])

## Inference time

In [ ]:
%%time
predict(df_test.enc_input.values[50],model2)

In [ ]:
# VALIDATION BELU SCORE
BLEU_w2v = []
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        pred = predict(str(i.enc_input),model2).split()
        act = [str(i.dec_output).split()]
        b =bleu.sentence_bleu(act,pred)
        BLEU_w2v.append(b)
    except:
        continue

In [ ]:
print("Triain BELU Score = ",np.mean(BLEU_w2v))

## Model 3 -EMBEDDINGS WITH  FAST TEXT

In [ ]:
!wget --header="Host: dl.fbaipublicfiles.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://fasttext.cc/" "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip" -c -O 'crawl-300d-2M-subword.zip'

In [ ]:
import io
import zipfile
from gensim.models import KeyedVectors
from gensim.models import FastText as fText

In [ ]:
z = zipfile.ZipFile("crawl-300d-2M-subword.zip" , 'r')
z.extractall("ft/")

In [ ]:
ft_model = fText.load_fasttext_format("/content/ft/crawl-300d-2M-subword",encoding='latin1')

In [ ]:
embedding_matrix_ft_input = np.zeros((len(tk_inp.word_index)+1,300))
for word,index in tqdm(tk_inp.word_index.items()) :
    try:
        vec = en_model[word]
        embedding_matrix_ft_input[index] = vec
    except:
        continue

In [ ]:
embedding_matrix_ft_out = np.zeros((len(tk_out.word_index)+1,300))
for word,index in tqdm(tk_out.word_index.items()) :
    try:
        vec = en_model[word]
        embedding_matrix_ft_out[index] = vec
    except:
        continue

## MODEL 3 ARCHTECTURE

In [ ]:
def encoder3(input_shape,vocab, emb_output, lstm_units, enc_input,embedding_matrix):
  
  enc_emb = layers.Embedding(vocab, emb_output,weights= [embedding_matrix] ,mask_zero = True,input_length=input_shape,trainable=False)(enc_input)
  enc_lstm , enc_state_h,enc_state_c = layers.LSTM(units= lstm_units,return_sequences=True,return_state=True)(enc_emb)
  return enc_lstm , enc_state_h,enc_state_c

def decoder3(input_shape,vocab, emb_output, lstm_units,enc_states, dec_input,embedding_matrix):
  
  dec_emb = layers.Embedding(vocab, emb_output ,weights= [embedding_matrix], mask_zero = True,input_length=input_shape,trainable=False)(dec_input)
  dec_lstm, dec_state_h,dec_state_c = layers.LSTM(units=lstm_units,return_sequences=True,return_state=True)(dec_emb,initial_state= enc_states)
  return dec_lstm, dec_state_h,dec_state_c

In [ ]:
enc_input3 = layers.Input(shape=(35))
enc_lstm3 , enc_state_h3,enc_state_c3 = encoder3(35,len(tk_inp.word_index)+1 , 300 ,256, enc_input3 ,embedding_matrix_ft_input)

dec_input3 = layers.Input(shape = (35))
dec_lstm3 , dec_state_h3,dec_state_c3 = decoder3(35,len(tk_out.word_index)+1 , 300 , 256 , [enc_state_h3,enc_state_c3],dec_input3,embedding_matrix_ft_out)

dense3 = layers.Dense(len(tk_out.word_index)+1,activation="softmax")(dec_lstm3)

model3  = Model(inputs=[enc_input3,dec_input3],outputs=dense3)

In [ ]:
callback3 =[ tf.keras.callbacks.ModelCheckpoint( "/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_ft/besh.h5",save_best_only=True,mode="min" ,save_weights_only=True),
           tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5,verbose=1,min_delta=0.0001)
]

train_steps = train_dataloader.__len__()
val_steps  = val_dataloader.__len__()

model3.compile(optimizer="adam",loss='sparse_categorical_crossentropy')

In [ ]:
history = model3.fit(train_dataloader,steps_per_epoch=train_steps,epochs=50,validation_data = val_dataloader,validation_steps =val_steps,callbacks=callback3)

In [ ]:
model3.load_weights("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_ft/besh.h5")

## PREDICTIONS FOR TEST DATASET

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[19])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[19],model3))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[19])

In [ ]:
print("INPUT SENTENCE ===> ",df_test.enc_input.values[50])
print("PREDICTED SENTENCE ===> ",predict(df_test.enc_input.values[50],model3))
print("ACTUAL SENTENCE ===> ",df_test.dec_output.values[50])

## Inference Time

In [ ]:
%%time
predict(df_test.enc_input.values[50],model3)

In [ ]:
# VALIDATION BELU SCORE
BLEU_val_ft = []
test_data = df_val.loc[np.random.choice(df_val.index,size = 2000)]
for ind,i in tqdm(test_data.iterrows(),position=0):
    pred = predict(str(i.enc_input),model3).split()
    act = [str(i.dec_output).split()]
    b =bleu.sentence_bleu(act,pred)
    BLEU_val_ft.append(b)

In [ ]:
print("Triain BELU Score = ",np.mean(BLEU_val_ft))

## Model Comparison

In [ ]:
s1 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/char_trainable_embedding/besh.h5")
s2 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_trainable_embedding/besh.h5")
s3 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_w2v/besh.h5")
s4 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/word_ft/besh.h5")
s5 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/bidirectional_train_emb/besh.h5")
s6 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/multi_layered_word/besh.h5")
s7 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_dot/besh.h5")
s8 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_gernal/best.h5")
s9 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/attention_concat/best.h5")
s10 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/monitonic_attention_dot/best.h5")
s11 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/monotonic_attention_general/best.h5")
s12 = os.path.getsize("/content/drive/MyDrive/ColabNotebooks/cs2/model_save/monotonic_attention_concat/best.h5")


In [ ]:
df_comp = pd.DataFrame()
df_comp["Model"] = ["Encoder Decoder(Char Level)","Encoder Decoder","Encoder Decoder","Encoder Decoder","Bidirectional Encoder Decoder","Multilayered Encoder Decoder","Attention Dot Model","Attention Gernal Model","Attention Concat Model","Monotonic Attention Dot Model","Monotonic Attention Gernal Model","Monotonic Attention Concat Model"]
df_comp["Embedding"] = ["One Hot Encoding","Trainable Embedding" , "Pretrained Word2Vec " ,"Fast Text","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding","Trainable Embedding"]
df_comp["BLEU Score(Greedy Search)"] = [0.3139,0.4603,0.4453,0.4569,0.4509,0.4527, 0.5055,0.5545,0.5388,0.5469,0.5514,0.5348]
df_comp["BLEU Score(Beam Search)"] = ["-","-","-","-",0.4561,0.4557,0.5411,0.5324,0.5671,"-","-","-"]
df_comp["Model Size(bytes)"] = [ s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12]
df_comp["Model Parameters"] = ["616,488	","26,363,578" , "8,158,378", "8,158,378","35,018,938" ," 28,464,826","33,353,914","33,419,706","33,485,755","33,353,914","33,419,706","33,485,755"]
df_comp["Inference Time(ms)"] = [143,92.3 , 94.5 , 93.7,250,311,157,164,189,164,179,176]
df_comp